<a href="https://colab.research.google.com/github/hannanabdul55/neural-nets-research-playground/blob/master/Ad_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import json
import re

import torch

# Try import ing external module, if not, pip install the pacage and import 
try:
    from transformers import BertModel, BertTokenizer
except:
    !pip install transformers
    from transformers import BertModel, BertTokenizer

import sklearn
from sklearn_pandas import DataFrameMapper
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

     |████████████████████████████████| 1.3MB 2.8MB/s 
     |████████████████████████████████| 2.9MB 15.9MB/s 
     |████████████████████████████████| 890kB 32.8MB/s 
     |████████████████████████████████| 1.1MB 24.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=e3ffc36bd097cd346820faaac6a95a8c8ce92294e7136105ce0fa0571b2672d4
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
# set the device to the used
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Dataset generation
This section can be skipped if you have the preprocessed data in your drive.  

First, we read the CSV file into a Pandas DataFrame.

In [ ]:
df = pd.read_csv("https://fbpac-images.s3.amazonaws.com/fbpac/en-US.csv.gz")

## Some useful information of the Dataframe

In [ ]:
print(f"Total number of ads: {df.shape[0]}")
print(f"Number of political ads: {(df['political'] >= 1).sum()}")
print(f"Percentage of political ads: {(df['political'] >= 1).sum()/df.shape[0]}")


targets = []

def update_targets(target):
    '''
        This method parsees the target JSON file and appends any new target to the targets list.
    '''
    if pd.isna(target):
        return
    target = json.loads(target)
    for t in target:
        if t['target'] not in targets:
            targets.append(t['target'])


# apply the update_targets method to every row of the DataFrame.
df.apply(lambda row: update_targets(row['targets']), axis=1)
print(f"possible values of targets: \n{targets}")

Total number of ads: 222168
Number of political ads: 44277
Percentage of political ads: 0.19929512801123475


## Preprocess
We want to parse the `target` column and use that information to construct additional feature columns ot every data based on the target information. 

In [ ]:
 def update_row_with_targets(row):
     if pd.isna(row['targets']):
         return row
     tgt = json.loads(row['targets'])
     for t in tgt:
         if 'segment' in t:
            row[t['target']] = t['segment']
     return row
 
 
 df = df.append(pd.DataFrame(columns=targets)).apply(lambda row: update_row_with_targets(row), axis=1)

# Preprocess dataset

In [ ]:
df.drop(columns=['Unnamed: 0','id','html', 'thumbnail', 'targeting', 'suppressed',
       'targets','thumbnail', 'created_at', 'updated_at', 'lang', 'images','listbuilding_fundraising_proba',
       'page_id', 'List','entities', 'page', 'lower_page','targetings', 'Mobile App'], inplace=True)

In [ ]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

df['message'] = df.apply(lambda row: cleanhtml(row['message']), axis=1)

# Save processed CSV file to drive
Please note that to do this, you will have to connect drive to this notebook. Then, mention the entire path (including file name) from the root of your drive. An example is given as the default. Use the form in the following cell to give the path.

In [ ]:
#@title File path if you want to save csv to Drive
path_in_drive = "UMass/datasets/processed_ad_dataset.csv" #@param {type:"string"}
full_path = f"/content/drive/My Drive/{path_in_drive}"
df.to_csv(full_path)

Great! You now have the entire preprocessed Ad dataset saved to your drive!

# [WIP]Modeling

In [ ]:
# Run this ONLY if you're starting the notebook from here. Please set teh drive path in the previous cell
df = pd.read_csv(full_path)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (27,28,31,32,33,42,45,46,47,48,49,50,51) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
sample_ids = tokenizer.encode('Hello this is Abdul')
print(f"Hello this is Abdul == {sample_ids}")

Hello this is Abdul == [101, 7592, 2023, 2003, 10298, 102]


In [ ]:
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
model.eval()

In [ ]:
def get_embedding(sent):
    toks = tokenizer.encode(sent)
    with torch.no_grad():
        out = model(input_ids=torch.LongTensor(toks).unsqueeze(0))
    hidden_states = out[2]
    sent_embed = torch.mean(hidden_states[-1], dim=1).squeeze()
    return sent_embed
